<a href="https://colab.research.google.com/github/AndresPautrat/IA_UPC/blob/main/Legendary_Pokemons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def add_bias_node(inputs):
    bias_node = -np.ones(len(inputs))
    return np.column_stack((inputs, bias_node))

class Perceptron(object):
    """
    A basic Perceptron
    Parameters
    ----------
    eta : float
        The learning rate of the network. Should be between
        0 and 1.
    """

    def __init__(self, eta):
        self.eta = eta

    def _initialize(self, inputs, targets):
        inputs = np.asarray(inputs)
        # gets set to 2d in _add_bias_node

        targets = np.asarray(targets)
        if targets.ndim == 1:
            targets = targets[:, None]

        self.targets = targets
        self.inputs = inputs

        # Set up network size
        self.m = inputs.shape[1]

        # Set up targets size
        self.n = targets.shape[1]

        # number of observations
        self.nobs = inputs.shape[0]

        # Initialise network weights plus 1 for bias node
        self._init_weights()

    def _init_weights(self):
        self.weights = np.random.rand(self.m+1, self.n)*0.1-0.05

    def fit(self, inputs, targets, max_iter):
        """
        Trains the network.
        Parameters
        ----------
        inputs : array-like
            The inputs data
        targets : array-like
            The targets to train on
        max_iter : int
            The number of iterations to perform
        """
        self._initialize(inputs, targets)
        inputs = self.inputs
        targets = self.targets
        weights = self.weights
        eta = self.eta

        # Add the inputs that match the bias node
        inputs = add_bias_node(inputs)
        # Training
        change = range(self.nobs)

        for n in range(max_iter):
            outputs = self.predict(inputs, weights, add_bias=False)
            weights += eta * np.dot(inputs.T, targets - outputs)

            # Randomize order of inputs
            np.random.shuffle(change)
            inputs = inputs[change, :]
            targets = targets[change, :]

        self.weights = weights

    def predict(self, inputs=None, weights=None, add_bias=True):
        """ Run the network forward """
        if weights is None:
            try:
                weights = self.weights
            except:
                raise ValueError("fit the classifier first "
                                 "or give weights")
        if inputs is None:
            inputs = self.inputs
        if add_bias:
            inputs = add_bias_node(inputs)

        outputs = np.dot(inputs, weights)

        # Threshold the outputs
        return np.where(outputs>0, 1, 0)


    def confusion_matrix(self, inputs, targets, weights, summary=True):
        """
        Confusion matrix
        If summary is True, returns percent correct, else returns
        the whole confusion matrix.
        """
        inputs = np.asarray(inputs)
        targets = np.asarray(targets)

        # Add the inputs that attach the bias node
        inputs = add_bias_node(inputs)

        outputs = np.dot(inputs, weights)

        n_classes = targets.ndim == 1 and 1 or targets.shape[1]

        if n_classes == 1:
            n_classes = 2
            outputs = np.where(outputs > 0, 1, 0)
        else:
            # 1-of-N encoding
            outputs = np.argmax(outputs, 1)
            targets = np.argmax(targets, 1)

        outputs = np.squeeze(outputs)
        targets = np.squeeze(targets)

        cm = np.histogram2d(targets, outputs, bins=n_classes)[0]
        if not summary:
            return cm
        else:
            return np.trace(cm)/np.sum(cm)*100

In [ ]:
def _linear_delta(targets, outputs, nobs):
    return (targets - outputs)/nobs

def _logistic_delta(targets, outputs, *args):
    return (targets - outputs)*outputs

def _softmax_delta(targets, outputs, nobs):
    return (targets - outputs)/nobs

_calc_deltao = {
        "linear" : _linear_delta,
        "logistic" : _logistic_delta,
        "softmax" : _softmax_delta
        }

def _linear_activation(outputs, *args):
    return outputs

def _logistic_activation(outputs, beta, *args):
    return 1/(1+np.exp(-beta*outputs))

def _softmax_activation(outputs, *args):
    # this is multinomial logit
    eX = np.exp(outputs)
    return eX/eX.sum(axis=1)[:,None]


_activation_funcs = {
        "linear" : _linear_activation,
        "logistic" : _logistic_activation,
        "softmax" : _softmax_activation,
        }

In [ ]:
class MLP(Perceptron):
    """
    A Multi-Layer Perceptron
    """
    def __init__(self, nhidden, eta, beta=1, momentum=0.9, outtype='logistic'):
        # Set up network size
        self.nhidden = nhidden
        self.eta = eta

        self.beta = beta
        self.momentum = momentum
        self.outtype = outtype


    def _init_weights(self):
        # Initialise network
        weights1 = np.random.rand(self.m+1, self.nhidden)-0.5
        weights1 *= 2/np.sqrt(self.m)
        weights2 = np.random.rand(self.nhidden+1,self.n)-0.5
        weights2 *= 2/np.sqrt(self.nhidden)

        self.weights1 = weights1
        self.weights2 = weights2

    def earlystopping(self, inputs, targets, valid_input, valid_target,
                            max_iter=100, epsilon=1e-3, disp=True):

        self._initialize(inputs, targets)
        valid_input = add_bias_node(valid_input)

        # 2 iterations ago, last iteration, current iteration
        # current iteration, last iteration, 2 iterations ago
        last_errors = [0, np.inf, np.inf]

        count = 0

        while np.any(np.diff(last_errors) > epsilon):
            count += 1
            if disp:
                print(count)

            # train the network
            self.fit(inputs, targets, max_iter, init=False, disp=disp)
            last_errors[2] = last_errors[1]
            last_errors[1] = last_errors[0]

            # check on the validation set
            valid_output = self.predict(valid_input, add_bias=False)
            errors = valid_target - valid_output
            last_errors[0] = 0.5*np.sum(errors**2)
        
        if disp:
            print("Stopped in %d iterations" % count, last_errors)
        return last_errors[0]

    def fit(self, inputs, targets, max_iter, disp=True, init=True):
        """
        Train the network

        Parameters
        ----------
        inputs : array-like
            The inputs data
        targets : array-like
            The targets to train on
        max_iter : int
            The number of iterations to perform
        init : bool
            Whether to initialize the weights or not.
        """
        if init:
            self._initialize(inputs, targets)
        inputs = self.inputs
        targets = self.targets
        weights1 = self.weights1
        weights2 = self.weights2
        eta = self.eta
        momentum = self.momentum
        nobs = self.nobs

        outtype = self.outtype

        # Add the inputs that match the bias node
        inputs = add_bias_node(inputs)
        change = list(range(self.nobs))

        updatew1 = np.zeros_like(weights1)
        updatew2 = np.zeros_like(weights2)


        for n in range(1, max_iter+1):

            # predict attaches hidden
            outputs = self.predict(inputs, add_bias=False)

            error = targets - outputs
            obj = .5 * np.sum(error**2)

            # Different types of output neurons
            deltao = _calc_deltao[outtype](targets, outputs, nobs)
            hidden = self.hidden
            deltah = hidden * (1. - hidden) * np.dot(deltao, weights2.T)

            updatew1 = (eta*(np.dot(inputs.T, deltah[:,:-1])) +
                        momentum*updatew1)
            updatew2 = (eta*(np.dot(self.hidden.T, deltao)) +
                        momentum*updatew2)
            weights1 += updatew1
            weights2 += updatew2

            # Randomise order of inputs
            np.random.shuffle(change)
            inputs = inputs[change,:]
            targets = targets[change,:]

        if disp:
            print("Iteration: ", n, " Objective: ", obj)

        # attach results
        self.weights1 = weights1
        self.weights2 = weights2
        self.outputs = outputs

    def predict(self, inputs=None, add_bias=True):
        """
        Run the network forward.
        """
        if inputs is None:
            inputs = self.inputs

        if add_bias:
            inputs = add_bias_node(inputs)

        hidden = np.dot(inputs, self.weights1)
        hidden = _activation_funcs["logistic"](hidden, self.beta)
        hidden = add_bias_node(hidden)
        self.hidden = hidden

        outputs = np.dot(self.hidden, self.weights2)

        outtype = self.outtype

        # Different types of output neurons
        return _activation_funcs[self.outtype](outputs, self.beta)

    def confusion_matrix(self, inputs, targets, summary=True):
        """
        Confusion matrix.
        """
        targets = np.asarray(targets)

        # Add the inputs that match the bias node
        inputs = add_bias_node(inputs)
        # predict attaches hidden
        outputs = self.predict(inputs, add_bias=False)

        n_classes = targets.ndim == 1 and 1 or targets.shape[1]

        if n_classes==1:
            n_classes = 2
            # 50% cut-off with continuous activation function
            outputs = np.where(outputs > 0.5, 1, 0)
        else:
            # 1-of-N encoding
            outputs = np.argmax(outputs, 1)
            targets = np.argmax(targets, 1)

        outputs = np.squeeze(outputs)
        targets = np.squeeze(targets)

        cm = np.histogram2d(targets, outputs, bins=n_classes)[0]

        if not summary:
            return cm
        else:
            return np.trace(cm)/np.sum(cm)*100

In [ ]:
X = X_all[0:10
          ]
X_num=[0 for i in range(len(X))]
X_num[2]=1.
X_num[3]=1.
and_clf = MLP(10, .25)

and_clf.fit(X, X_num, 10000)
and_clf.predict(X)

Iteration:  10000  Objective:  0.9999999998775636


array([[4.41489571e-11],
       [4.94170381e-11],
       [5.75814071e-11],
       [6.48550149e-11],
       [4.41476262e-11],
       [5.00608285e-11],
       [5.92092891e-11],
       [6.75131895e-11],
       [6.77812568e-11],
       [4.34660946e-11],
       [4.88895650e-11]])

In [ ]:
X = X_all[0:10]
X_num = [0, 0, 1, 1,0,0,0,0,0,0]
and_clf = MLP(7, .25)

In [ ]:
X_sup[30:40]

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1.0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1.0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 1.0, 0, 0, 0, 0]]

In [ ]:
X_num

[0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0]

In [ ]:
df = pd.read_csv("Pokemon.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
1,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
4,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False


In [ ]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,801,801,801,415,801,801,801,801,801,801,801,801,801
unique,722,801,19,19,201,95,112,104,106,93,109,7,3
top,479,Galvantula,Water,Flying,600,60,100,70,60,80,50,1,False
freq,6,1,112,97,37,67,40,54,51,52,46,166,735


In [ ]:
X_all= df.iloc[1:,[4,5,6,7,8,9,10]].values

In [ ]:
X_all=X_all.astype(float)

In [ ]:
X_all=X_all/1000

In [ ]:
X_all[20:30]

array([[0.251, 0.04 , 0.045, 0.04 , 0.035, 0.035, 0.056],
       [0.349, 0.063, 0.06 , 0.055, 0.05 , 0.05 , 0.071],
       [0.479, 0.083, 0.08 , 0.075, 0.07 , 0.07 , 0.101],
       [0.579, 0.083, 0.08 , 0.08 , 0.135, 0.08 , 0.121],
       [0.253, 0.03 , 0.056, 0.035, 0.025, 0.035, 0.072],
       [0.413, 0.055, 0.081, 0.06 , 0.05 , 0.07 , 0.097],
       [0.262, 0.04 , 0.06 , 0.03 , 0.031, 0.031, 0.07 ],
       [0.442, 0.065, 0.09 , 0.065, 0.061, 0.061, 0.1  ],
       [0.288, 0.035, 0.06 , 0.044, 0.04 , 0.054, 0.055],
       [0.438, 0.06 , 0.085, 0.069, 0.065, 0.079, 0.08 ]])

In [ ]:
num_poke=[0 for i in range(len(X_all))]
for i in range(len(num_poke)):
  if (df[12][i]=="True"):
    num_poke[i]=1
    print(X_all[i])

[0.58  0.09  0.09  0.085 0.125 0.09  0.1  ]
[0.58  0.09  0.1   0.09  0.125 0.085 0.09 ]
[0.3   0.041 0.064 0.045 0.05  0.05  0.05 ]
[0.78  0.106 0.19  0.1   0.154 0.1   0.13 ]
[0.78  0.106 0.15  0.07  0.194 0.12  0.14 ]
[0.6 0.1 0.1 0.1 0.1 0.1 0.1]
[0.58  0.115 0.115 0.085 0.09  0.075 0.1  ]
[0.58  0.1   0.075 0.115 0.09  0.115 0.085]
[0.3   0.05  0.064 0.05  0.045 0.05  0.041]
[0.68  0.106 0.13  0.09  0.11  0.154 0.09 ]
[0.6 0.1 0.1 0.1 0.1 0.1 0.1]
[0.58 0.08 0.05 0.1  0.1  0.2  0.05]
[0.58  0.08  0.075 0.15  0.075 0.15  0.05 ]
[0.6  0.08 0.08 0.09 0.11 0.13 0.11]
[0.7  0.08 0.1  0.12 0.14 0.15 0.11]
[0.6  0.08 0.09 0.08 0.13 0.11 0.11]
[0.7  0.08 0.13 0.1  0.16 0.12 0.11]
[0.67 0.1  0.1  0.09 0.15 0.14 0.09]
[0.77 0.1  0.15 0.09 0.18 0.16 0.09]
[0.67 0.1  0.15 0.14 0.1  0.09 0.09]
[0.77 0.1  0.18 0.16 0.15 0.09 0.09]
[0.68  0.105 0.15  0.09  0.15  0.09  0.095]
[0.78  0.105 0.18  0.1   0.18  0.1   0.115]
[0.6 0.1 0.1 0.1 0.1 0.1 0.1]
[0.6  0.05 0.15 0.05 0.15 0.05 0.15]
[0.6  0.05 0

In [ ]:
and_clf = MLP(7, .25)
and_clf.fit(X_sup, num_poke, 1)

Iteration:  1  Objective:  50.24916285047452


In [ ]:
predictions=and_clf.predict(X_all)
for i in range(len(num_poke)):
  if (df[12][i]=="True"):
    print(predictions[i])

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


In [ ]:
X_sup=[[0 for j in range(len(X_all[0]))] for i in range(len(X_all))]
X_sup[0][0]

0

In [ ]:
for i in range(len(X_all)):
    if(X_all[i,0]>500.):
      X_sup[i][0]=1.
    for j in range(1,len(X_all[0])):
      if(X_all[i,j]>100.):
        X_sup[i][j]=1.




In [ ]:
X_sup

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 0, 1.0, 1.0, 1.0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 0, 0, 1.0, 0, 0],
 [1.0, 0, 1.0, 1.0, 1.0, 0, 0],
 [1.0, 0, 1.0, 0, 1.0, 1.0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 0, 0, 0, 1.0, 0],
 [1.0, 0, 1.0, 1.0, 1.0, 1.0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1.0, 0, 0, 0, 1.0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1.0],
 [1.0, 0, 0, 0, 1.0, 0, 1.0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1.0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1.0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1.0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [

In [ ]:
F=[0,1,2,3,4,5,6,7,8,9]

F[2]

TypeError: ignored

In [ ]:
import tkinter as tk

In [ ]:
ventana=tk.Tk()
ventana.geometry('380x380')
ventana.configure(background='dark turquoise')

ventana.mainloop()

TclError: ignored